### Step backward feature selection

Step Backward Feature Selection starts by fitting a model using all features in the data set and determining its performance. 

Then, it trains models on all possible combinations of all features -1, and removes the feature that returns the model with the lowest performance.

In the third step it trains models in all possible combinations of the features remaining from step 2 -1 feature, and removes the feature that produced the lowest performing model.

The algorithm stops on a criteria determined by the user. This criteria could be that the model performance does not decrease beyond a certain threshold, or alternatively, as in the mlxtend implementation, when we reach a certain number of selected features.

The evaluation metric can be the roc_auc for classification or the r squared for regression for example, and is determined by the user.

Step Backward Feature Selection is called greedy, because it evaluates all possible n, and then n-1 and n-2 and so on feature combinations. Therefore, it is very computationally expensive, and sometimes, if feature space is big, even unfeasible.

There is a special package in Python that implements this type of feature selection: mlxtend.
http://rasbt.github.io/mlxtend/

In the mlxtend implementation of the Step Backward Feature Selection, the stopping criteria is an arbitrarily set number of features. So the search will finish when we reach the desired number of selected features. 

This is somewhat arbitrary, we might be selecting a sub-opimal number of features, or likewise, a high number of features. But, by looking at the performance metric returned by the algorithm as it selects the features, we can have a view, if more features do add value, or not. 


**Note**
If we wanted to stop the search by using another criteria, we would have to code the algorithm ourselves, unfortunately :(

Here I will use the Step Backward Feature Selection algorithm from mlxtend in a classification and regression dataset.

In [1]:
# import libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [2]:
# load data

data = pd.read_csv('..\precleaned-datasets\dataset_2.csv')
data.head()

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109
0,4.532710,3.280834,17.982476,4.404259,2.349910,0.603264,2.784655,0.323146,12.009691,0.139346,...,2.079066,6.748819,2.941445,18.360496,17.726613,7.774031,1.473441,1.973832,0.976806,2.541417
1,5.821374,12.098722,13.309151,4.125599,1.045386,1.832035,1.833494,0.709090,8.652883,0.102757,...,2.479789,7.795290,3.557890,17.383378,15.193423,8.263673,1.878108,0.567939,1.018818,1.416433
2,1.938776,7.952752,0.972671,3.459267,1.935782,0.621463,2.338139,0.344948,9.937850,11.691283,...,1.861487,6.130886,3.401064,15.850471,14.620599,6.849776,1.098210,1.959183,1.575493,1.857893
3,6.020690,9.900544,17.869637,4.366715,1.973693,2.026012,2.853025,0.674847,11.816859,0.011151,...,1.340944,7.240058,2.417235,15.194609,13.553772,7.229971,0.835158,2.234482,0.946170,2.700606
4,3.909506,10.576516,0.934191,3.419572,1.871438,3.340811,1.868282,0.439865,13.585620,1.153366,...,2.738095,6.565509,4.341414,15.893832,11.929787,6.954033,1.853364,0.511027,2.599562,0.811364


In [3]:
data.shape

(50000, 109)

In [3]:
# split the data into train and test
# separate train and test sets

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((35000, 108), (15000, 108))

### Remove Correlated features

Step Forward Feature Selection takes a long time to run, so to speed it up we will reduce the feature space by removing correlated features first.

In [4]:
# remove correlated features to reduce the feature space

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  36


In [5]:
# remove correlated features
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((35000, 72), (15000, 72))

### Step Backward Feature Selection

For the Step Backward feature selection algorithm, we are going to use the class SFS from MLXtend:
http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/

In [6]:
# within the SFS we indicate:

# 1) the algorithm we want to create, in this case RandomForests
# (note that I use few trees to speed things up)

# 2) the stopping criteria: want to select 10 features 

# 3) wheter to perform step forward or step backward

# 4) the evaluation metric: in this case the roc_auc
# 5) the cross-validation

sfs = SFS(estimator=RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=2),
          k_features=65, # the more features we want, the longer it will take to run
          forward=False,  # for Step forward selection 
          floating=False,
          verbose=2, # this indicates how much message to print out during intermediate steps
          scoring='roc_auc',
          cv=3)

sfs.fit(np.array(X_train), y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:  5.3min finished

[2022-06-02 12:30:07] Features: 71/65 -- score: 0.6327122738842678[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  71 out of  71 | elapsed:  5.5min finished

[2022-06-02 12:35:36] Features: 70/65 -- score: 0.6353304737101858[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:  5.8min finished

[2022-06-02 12:41:27] Features: 69/65 -- score: 0.6305156828055708[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Don

SequentialFeatureSelector(cv=3,
                          estimator=RandomForestClassifier(n_estimators=10,
                                                           n_jobs=2,
                                                           random_state=0),
                          forward=False, k_features=65, scoring='roc_auc',
                          verbose=2)

From the output above, we can see that after adding the 8th feature, the performance begins to plateau. Adding the 9th and 10th feature did not increase the performance.

If instead of selecting 10 features, we select more as the stopping criteria, we could have a clearer view of the progression of the performance vs number of features.

In [11]:
# these are the index of the selected features
sfs.k_feature_names_

('0', '14', '15', '19', '34', '43', '47', '52', '64', '68')

In [13]:
# indices of the selected features
sfs.k_feature_idx_

(0, 14, 15, 19, 34, 43, 47, 52, 64, 68)

In [17]:
# selected features names
selected_features = X_train.columns[list(sfs.k_feature_idx_)]

In [18]:
selected_features

Index(['var_1', 'var_15', 'var_16', 'var_21', 'var_45', 'var_55', 'var_62',
       'var_69', 'var_91', 'var_98'],
      dtype='object')

## Compare performance of feature subsets

In [19]:
# function to train random forests and evaluate the performance

def run_randomForests(X_train, X_test, y_train, y_test):
    
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)

    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

In [21]:
# Original data performance
run_randomForests(X_train, X_test, y_train, y_test)

Train set
Random Forests roc-auc: 0.7119921185820277
Test set
Random Forests roc-auc: 0.6957598691250635


In [22]:
# Selected features data performance
run_randomForests(X_train[selected_features], X_test[selected_features], y_train, y_test)

Train set
Random Forests roc-auc: 0.7129560989521158
Test set
Random Forests roc-auc: 0.7027726779668045


As you see, in this dataset, with 10 features we obtain a similar performance than that obtained using all variables in the dataset.

## Regression

Let's now repeat the process but in the context of regression. With the house prices dataset from Kaggle, the aim is to predict the continuous target: House Price.

In [25]:
# load dataset
data = pd.read_csv('house_price.csv')
data.shape

(1460, 81)

In [26]:
# In practice, feature selection should be done after data pre-processing,
# so ideally, all the categorical variables are encoded into numbers,
# and then you can assess how deterministic they are of the target

# here for simplicity I will use only numerical variables
# select numerical columns:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [27]:
# separate train and test sets

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((1022, 37), (438, 37))

## Remove Correlated features

In [28]:
# find and remove correlated features

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  3


In [29]:
# removed correlated features
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((1022, 34), (438, 34))

In [30]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [31]:
# step forward feature selection

sfs = SFS(RandomForestRegressor(n_estimators=10, n_jobs=2, random_state=10), 
           k_features=20, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='r2',
           cv=2)

sfs = sfs.fit(np.array(X_train), y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:    4.1s finished

[2022-06-02 11:24:05] Features: 1/20 -- score: 0.6448864917335085[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    1.5s finished

[2022-06-02 11:24:06] Features: 2/20 -- score: 0.6946490592888616[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    1.3s finished

[2022-06-02 11:24:08] Features: 3/20 -- score: 0.732141233157488[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   

From the logs above, we see that after ~17 features, adding more features does not really improve performance.

In [32]:
sfs.k_feature_idx_

(1, 3, 4, 5, 6, 7, 11, 12, 13, 14, 16, 17, 18, 19, 23, 24, 25, 28, 29, 30)

In [33]:
# get the features 
X_train.columns[list(sfs.k_feature_idx_)]

Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'Fireplaces',
       'GarageCars', 'WoodDeckSF', '3SsnPorch', 'ScreenPorch', 'PoolArea'],
      dtype='object')

In [36]:
selected_feats = X_train.columns[list(sfs.k_feature_idx_)]

# compare performance of the feature sets

In [34]:
# function to train random forests and evaluate the performance

def run_randomForests(X_train, X_test, y_train, y_test):
    
    rf = RandomForestRegressor(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)

    print('Train set')
    pred = rf.predict(X_train)
    print('Random Forests roc-auc: {}'.format(r2_score(y_train, pred)))
    
    print('Test set')
    pred = rf.predict(X_test)
    print('Random Forests roc-auc: {}'.format(r2_score(y_test, pred)))

In [37]:
# Selected features
run_randomForests(X_train[selected_feats], X_test[selected_feats], y_train, y_test)

Train set
Random Forests roc-auc: 0.8629621682457452
Test set
Random Forests roc-auc: 0.8246015925536159


In [38]:
# Original
run_randomForests(X_train, X_test, y_train, y_test)

Train set
Random Forests roc-auc: 0.8699152317492538
Test set
Random Forests roc-auc: 0.8190809813112794


We see that the algorithm with 20 features performs as well as that with 24 features.